## Authorization to the DEDL services

In [1]:
from dedllogin import DEDL_auth
import requests
import json
import os
from getpass import getpass
from IPython.display import JSON

username = input("Please input your DESP username or email: ")
password = getpass("Please input your DESP password: ")

auth = DEDL_auth(username, password)

access_token = auth.get_token()
if access_token is not None:
    print("DEDL/DESP Access Token Obtained Successfully")
else:
    print("Failed to Obtain DEDL/DESP Access Token")

auth_headers = {"Authorization": f"Bearer {access_token}"}

Please input your DESP username or email:  jesper.dejby@ri.se
Please input your DESP password:  ········


DEDL/DESP Access Token Obtained Successfully


## Download data

We are aiming to download the Sentinel-3 SLSTR Level 2 Sea Surface Temperature dataset described here: https://hda.data.destination-earth.eu/ui/dataset/EO.EUM.DAT.SENTINEL-3.SL_2_WST___

In [2]:
response_sst = requests.post("https://hda.data.destination-earth.eu/stac/search", headers=auth_headers, json={
    "collections": ["EO.EUM.DAT.SENTINEL-3.SL_2_WST___"],
    "datetime": "2024-06-02T00:00:00Z/2024-06-03T00:00:00Z",
    'bbox': [53, 10, 66, 30] #latitude and longitude for Baltic Sea
})

In [3]:
from IPython.display import JSON

product_sst = response_sst.json()["features"][0]

print("Sea surface temprature")
JSON(product_sst)

Sea surface temprature


<IPython.core.display.JSON object>

In [4]:
from tqdm import tqdm
import time

# Define a list of assets to download
assets = ["20240601231931-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240603064235-v02.0-fv01.0.nc"]


for asset in assets:
    download_url = product_sst["assets"][asset]["href"]
    print(download_url)
    filename = asset
    print(filename)
    response = requests.get(download_url, headers=auth_headers)
    total_size = int(response.headers.get("content-length", 0))

    print(f"downloading {filename}")

    with tqdm(total=total_size, unit="B", unit_scale=True) as progress_bar:
        with open(filename, 'wb') as f:
            for data in response.iter_content(1024):
                progress_bar.update(len(data))
                f.write(data)

https://hda.data.destination-earth.eu/stac/collections/EO.EUM.DAT.SENTINEL-3.SL_2_WST___/items/S3B_SL_2_WST____20240601T231931_20240602T010030_20240603T064235_6059_093_329______MAR_O_NT_003/download/20240601231931-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240603064235-v02.0-fv01.0.nc?provider=dedl
20240601231931-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240603064235-v02.0-fv01.0.nc
downloading 20240601231931-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240603064235-v02.0-fv01.0.nc


675MB [00:01, 625MB/s] 


## Read data

We read the data and subset it too the baltic sea region.

In [53]:
import rioxarray
import matplotlib.pyplot as plt
import xarray as xr
import numpy as np
import netCDF4 as nc

The load_to_daskarray and load_to_array functions were early attempts to read the data. We later discovered better ways of handling the datasets.

In [136]:
def load_to_daskarray(nc_filename):
    grid_reduce = 5
    band_vars = xr.open_mfdataset(nc_filename)
    band_dict = {}
    
    for band_var in band_vars:
        print(f"Reading:  {band_var}")
        #band_dict[band_var] = band_vars[band_var].data[::grid_reduce,::grid_reduce]
        band_dict[band_var] = band_vars[band_var][:]
    band_vars.close()
    return band_dict

In [137]:
def load_to_array(nc_filename):
    nc_obj = nc.Dataset(nc_filename)
    band_dict = {}
    
    for band_var in nc_obj.variables.keys():
        print(f"Reading:  {band_var}")
        band_dict[band_var] = nc_obj[band_var][:].data
    band_vars.close()
    return band_dict

In [138]:
ds=load_to_array("20240601231931-MAR-L2P_GHRSST-SSTskin-SLSTRB-20240603064235-v02.0-fv01.0.nc")

Reading:  adi_dtime_from_sst
Reading:  aerosol_dynamic_indicator
Reading:  brightness_temperature
Reading:  dt_analysis
Reading:  dual_nadir_sst_difference
Reading:  l2p_flags
Reading:  lat
Reading:  lon
Reading:  nadir_sst_theoretical_uncertainty
Reading:  nedt
Reading:  probability_cloud_single_in
Reading:  probability_cloud_single_io
Reading:  quality_level
Reading:  satellite_zenith_angle
Reading:  sea_ice_fraction
Reading:  sea_ice_fraction_dtime_from_sst
Reading:  sea_surface_temperature
Reading:  sses_bias
Reading:  sses_standard_deviation
Reading:  sst_algorithm_type
Reading:  sst_dtime
Reading:  sst_theoretical_uncertainty
Reading:  time
Reading:  wind_speed
Reading:  wind_speed_dtime_from_sst


The normalise_image function was originally taken from the eumartools package and slightly adopted to handle our use-case instead of RGB image arrays.

In [139]:
# function from eumartool (https://gitlab.com/benloveday/eumartools/-/blob/main/eumartools/image_tools.py)
# contrast=[1.0, 1.0, 1.0]
def normalise_image(grid_array, contrast=False, unhitch=False):
    """Function to normalise an image using independant channels (unhitch=True)
        or combined channels (unhitch=False, default)

    Args:
        grid_array (numpy array): three-dimensional RGB array to be normalised
        contrast (list): non-linear (^2) channel scaling, e.g. [1.0 1.0 1.0]
        unhitch (bool): switch for independant or cross-channel normalisation

    Returns:
        if successful, the normalised array
        else returns an error
    """

    try:
        img_array = grid_array.copy()
        
        if unhitch:
            # normalise with separating channels
            for ii in range(a):
                minval = np.nanmin(img_array[:, :, ii])
                maxval = np.nanmax(img_array[:, :, ii])
                img_array[:, :, ii] = (img_array[:, :, ii] - minval) / (
                    maxval - minval
                )
                # apply contrast
                if contrast:
                    img_array[:, :, ii] = img_array[:, :, ii] ** contrast[ii]
        else:
            # normalise without separating channels
            minval = np.nanmin(img_array)
            maxval = np.nanmax(img_array)
            for ii in range(np.shape(img_array)[-1]):
                img_array[:, :, ii] = (img_array[:, :, ii] - minval) / (
                    maxval - minval
                )
                # apply contrast
                if contrast:
                    img_array[:, :, ii] = img_array[:, :, ii] ** contrast[ii]

            # non-linearity: contrast - note that the range is not between
            # 0 and 1, so need to renormalise afterwards
            if contrast:
                minval = np.nanmin(img_array)
                maxval = np.nanmax(img_array)
                for ii in range(np.shape(img_array)[-1]):
                    img_array[:, :, ii] = (img_array[:, :, ii] - minval) / (
                        maxval - minval
                    )

        return img_array
    except Exception as error:
        msg = "Unsuccessful!", error, "occurred."
        print(msg)
        return msg

## Final remarks

We initially thought that we could do all pre-processing (e.g. normalising) by accessing the pixel value array and performing operations directly on the array. However, we quickly realised that we needed to clip the raster data to our region and resample first. Hence, we moved away from this approach and did our pre-processing as outlined in the main.py and split_tif_files.py code. Normalisation and configuring the CNN model is outlined in CNN_SSC.ipynb.

In [140]:
norm=normalise_image(ds['sea_surface_temperature'])